In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.multiclass import unique_labels

# === Configuration ===
dataset_path = "leapGestRecog/leapGestRecog/00" 
img_size = 64
limit_per_class = 1000
pca_components = 100

X = []
y = []

# === Detect valid class folders ===
class_labels = sorted([
    folder for folder in os.listdir(dataset_path)
    if os.path.isdir(os.path.join(dataset_path, folder))
])
label_map = {label: idx for idx, label in enumerate(class_labels)}

print("📥 Loading gesture images (from nested subfolders)...")

# === Recursively load images from nested folders ===
for label in class_labels:
    class_path = os.path.join(dataset_path, label)
    count = 0

    for root, _, files in os.walk(class_path):
        for img_name in files:
            try:
                img_path = os.path.join(root, img_name)
                img = cv2.imread(img_path)
                if img is None:
                    continue
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (img_size, img_size))
                X.append(img.flatten())
                y.append(label_map[label])
                count += 1
                if count >= limit_per_class:
                    break
            except:
                continue
        if count >= limit_per_class:
            break

print(f"✅ Loaded {len(X)} images across {len(class_labels)} classes.")

# === Convert to NumPy arrays ===
X = np.array(X)
y = np.array(y)

# === Continue only if images were loaded ===
if len(X) > 0:
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # Apply PCA
    print("🔄 Applying PCA...")
    pca = PCA(n_components=pca_components)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    print("✅ PCA applied.")

    # Train the SVM model
    print("🚀 Training SVM...")
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)
    print("✅ SVM training complete.")

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # ✅ Fix: Filter labels and class names for report
    present_labels = unique_labels(y_test, y_pred)
    present_class_names = [class_labels[i] for i in present_labels]

    print("\n📊 Classification Report:")
    print(classification_report(y_test, y_pred, labels=present_labels, target_names=present_class_names))
    print(f"🎯 Accuracy: {accuracy * 100:.2f}%")

else:
    print("❌ No images were loaded. Please check your dataset structure.")


📥 Loading gesture images (from nested subfolders)...
✅ Loaded 2000 images across 11 classes.
🔄 Applying PCA...
✅ PCA applied.
🚀 Training SVM...
✅ SVM training complete.

📊 Classification Report:
               precision    recall  f1-score   support

      01_palm       1.00      1.00      1.00        40
         02_l       1.00      1.00      1.00        40
      03_fist       1.00      1.00      1.00        40
04_fist_moved       1.00      1.00      1.00        40
     05_thumb       1.00      1.00      1.00        40
     06_index       1.00      1.00      1.00        40
        07_ok       1.00      1.00      1.00        40
08_palm_moved       1.00      1.00      1.00        40
         09_c       1.00      1.00      1.00        40
      10_down       1.00      1.00      1.00        40

     accuracy                           1.00       400
    macro avg       1.00      1.00      1.00       400
 weighted avg       1.00      1.00      1.00       400

🎯 Accuracy: 100.00%
